In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import re
# import optuna
# from optuna.integration import XGBoostPruningCallback
sns.set_theme(style="darkgrid")

In [46]:
train_df = pd.read_csv('standard_dataset.csv')

In [47]:
train_df

,Unnamed: 0,area,floor,year_of_completion,price_log,gu_ec_price,top10_brand_ec2_price,top20_danji_ec2_price,transaction_year_month_ec
0,0,79.97,4,1987,11.775297,1.000000,0.079817,0.154972,0
1,1,79.97,2,1987,11.669938,1.000000,0.079817,0.154972,0
2,2,79.97,1,1987,11.775297,1.000000,0.079817,0.154972,0
3,3,79.97,2,1987,11.845827,1.000000,0.079817,0.154972,2
4,4,54.98,5,1987,11.585255,1.000000,0.079817,0.154972,3
...,...,...,...,...,...,...,...,...,...
240843,240916,67.57,2,2003,10.165890,0.030539,0.079817,0.087630,31
240844,240917,95.94,6,2007,10.691968,0.030539,0.079817,0.110988,30
240845,240918,100.17,5,2007,10.911464,0.030539,0.079817,0.110988,35
240846,240919,77.71,5,2007,10.596660,0.030539,0.079817,0.110988,32


In [48]:
train_df.drop('Unnamed: 0',axis = 1, inplace=True)

In [49]:
train_df.drop_duplicates(inplace=True)
train_df

,area,floor,year_of_completion,price_log,gu_ec_price,top10_brand_ec2_price,top20_danji_ec2_price,transaction_year_month_ec
0,79.97,4,1987,11.775297,1.000000,0.079817,0.154972,0
1,79.97,2,1987,11.669938,1.000000,0.079817,0.154972,0
2,79.97,1,1987,11.775297,1.000000,0.079817,0.154972,0
3,79.97,2,1987,11.845827,1.000000,0.079817,0.154972,2
4,54.98,5,1987,11.585255,1.000000,0.079817,0.154972,3
...,...,...,...,...,...,...,...,...
240842,106.21,6,2003,10.723289,0.030539,0.079817,0.087630,26
240843,67.57,2,2003,10.165890,0.030539,0.079817,0.087630,31
240844,95.94,6,2007,10.691968,0.030539,0.079817,0.110988,30
240845,100.17,5,2007,10.911464,0.030539,0.079817,0.110988,35


# Linear Regression

In [50]:
# train_X = train_df[train_df.columns.difference(['price_log'])]
# train_y = train_df['price_log']

data = train_df[train_df.columns.difference(['price_log'])]
target = train_df['price_log']

In [51]:
train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2, shuffle=True, random_state=34)

In [52]:
reg = LinearRegression().fit(train_x, train_y)
reg.score(test_x, test_y)

0.70305711305937

In [53]:
model = LinearRegression()
model.fit(train_x, train_y)

print("가중치(계수, 기울기 파라미터 W) :", model.coef_)
print("편향(절편 파라미터 b) :", model.intercept_)

print("훈련세트 점수: {:.2f}".format( model.score(train_x, train_y) ))
print("테스트세트 점수: {:.2f}".format( model.score(test_x, test_y) ))

가중치(계수, 기울기 파라미터 W) : [ 0.00971996  0.00903102  1.07385127  2.17903055  0.14605621  0.01201439
 -0.00247973]
편향(절편 파라미터 b) : 14.43752480952187
훈련세트 점수: 0.71
테스트세트 점수: 0.70


# timeseriessplit

In [11]:
train_timeseries_df = train_df.set_index('transaction_year_month_ec')

In [12]:
train_timeseries_df.sort_index(inplace=True)
train_timeseries_df

,area,floor,year_of_completion,price_log,gu_ec_price,top10_brand_ec2_price,top20_danji_ec2_price
transaction_year_month_ec,,,,,,,
0,79.9700,4,1987,11.775297,1.000000,0.079817,0.154972
0,89.2000,5,2002,10.545368,0.147771,0.079817,0.087630
0,49.6000,13,2005,10.118639,0.147771,0.079817,0.087630
0,84.9200,2,2005,10.434145,0.147771,0.079817,0.087630
0,84.8900,8,2005,10.442930,0.147771,0.079817,0.087630
...,...,...,...,...,...,...,...
35,59.4000,10,1997,11.667370,0.293756,0.079817,0.087630
35,64.4500,8,2005,10.725490,0.066155,0.079817,0.087630
35,64.4500,18,2005,10.857093,0.066155,0.079817,0.087630


In [13]:
train_timeseries_df.reset_index(inplace=True)

In [14]:
train_timeseries_df.drop('index', axis=1,inplace=True)

KeyError: "['index'] not found in axis"

In [54]:
train_timeseries_df

,transaction_year_month_ec,area,floor,year_of_completion,price_log,gu_ec_price,top10_brand_ec2_price,top20_danji_ec2_price
0,0,79.9700,4,1987,11.775297,1.000000,0.079817,0.154972
1,0,89.2000,5,2002,10.545368,0.147771,0.079817,0.087630
2,0,49.6000,13,2005,10.118639,0.147771,0.079817,0.087630
3,0,84.9200,2,2005,10.434145,0.147771,0.079817,0.087630
4,0,84.8900,8,2005,10.442930,0.147771,0.079817,0.087630
...,...,...,...,...,...,...,...,...
236592,35,59.4000,10,1997,11.667370,0.293756,0.079817,0.087630
236593,35,64.4500,8,2005,10.725490,0.066155,0.079817,0.087630
236594,35,64.4500,18,2005,10.857093,0.066155,0.079817,0.087630
236595,35,77.0731,8,2019,11.711785,0.293756,0.228032,0.228032


In [55]:
X_train = train_timeseries_df[train_timeseries_df['transaction_year_month_ec'] < 30].drop(['price_log'], axis=1) 
y_train = train_timeseries_df[train_timeseries_df['transaction_year_month_ec'] < 30]['price_log'] 
X_test = train_timeseries_df[train_timeseries_df['transaction_year_month_ec'] >= 30].drop(['price_log'], axis=1)
y_test = train_timeseries_df[train_timeseries_df['transaction_year_month_ec'] >= 30]['price_log']

In [25]:
X_train.set_index('transaction_year_month_ec', inplace=True)
# y_train.set_index('transaction_year_month_ec')
X_test.set_index('transaction_year_month_ec', inplace=True)

In [56]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((198695, 7), (198695,), (37902, 7), (37902,))

In [77]:
from sklearn.linear_model import LinearRegression 
from sklearn.neural_network import MLPRegressor 
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.svm import SVR 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import cross_val_predict 
from sklearn.metrics import r2_score


def RMSE(y, y_pred):
    rmse = mean_squared_error(y, y_pred) ** 0.5
    return rmse

# Spot Check Algorithms 
models = [] 
models.append(('LR', LinearRegression())) 
# models.append(('NN', MLPRegressor(solver = 'lbfgs'))) #neural network 
# models.append(('KNN', KNeighborsRegressor()))
# models.append(('DTree',DecisionTreeRegressor(max_depth=6, min_samples_split=10, min_samples_leaf=15, random_state=1))) 
models.append(('RF', RandomForestRegressor(n_estimators = 10))) # Ensemble method - collection of many decision trees
# models.append(('RF2', RandomForestRegressor(n_estimators=10, criterion='mse', max_depth=9, min_samples_split=50, min_samples_leaf=5, random_state=1, n_jobs=-1)))
# models.append(('ridge', Ridge(alpha=0.08, random_state=1)))
# models.append(('lasso', Lasso(alpha = 0.001, random_state=1)))
# models.append(('Enet', ElasticNet(alpha=0.003, l1_ratio=0.01, random_state=1)))
# models.append(('model_xgb', xgb.XGBRegressor(n_estimators=500, max_depth=9, min_child_weight=5, gamma=0.1, n_jobs=-1)))
models.append(('model_lgb', lgb.LGBMRegressor(n_estimators=500, max_depth=9, min_child_weight=5, n_jobs=-1)))

# models.append(('SVR', SVR(gamma='auto'))) # kernel = linear
# Evaluate each model in turn 
results = []
rmse_result = [] 
names = [] 
for name, model in models: 
    # TimeSeries Cross validation 
    tscv = TimeSeriesSplit(n_splits=15) 
    cv_results_temp = cross_val_score(model, X_train, y_train, cv=tscv, scoring='neg_mean_squared_error')
    cv_results = np.sqrt(-cv_results_temp)
    clf = model.fit(X_train, y_train)
    pred = clf.predict(X_test)
    rmse = RMSE(y_test, pred) 
    results.append(cv_results)
    rmse_result.append(rmse)
    names.append(name) 
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    print(rmse_result)
    # print(results)

# print(clf)
    

LR: 0.327211 (0.023110)
[0.38183204911017965]
RF: 0.142809 (0.012109)
[0.38183204911017965, 0.19107186532633508]
model_lgb: 0.165161 (0.008333)
[0.38183204911017965, 0.19107186532633508, 0.20697722590824688]


In [ ]:
# def RMSE(y, y_pred):
#     rmse = mean_squared_error(y, y_pred) ** 0.5
#     return rmse

# def rmse_cv(model):
#     # cv별로 학습하는 함수
#     tscv = TimeSeriesSplit(n_splits=10)
#     rmse_list = []
#     model_name = model.__class__.__name__
#     for _, (train_index, test_index) in tqdm(enumerate(tscv.split(train_X), start=1), desc=f'{model_name} Cross Validations...', total=10):
#         X_train, X_test = train_X.iloc[train_index], train_X.iloc[test_index]
#         y_train, y_test = train_y.iloc[train_index], train_y.iloc[test_index]
#         clf = model.fit(X_train, y_train)
#         pred = clf.predict(X_test)
#         rmse = RMSE(y_test, pred) 
#         rmse_list.append(rmse)
#     return model_name, rmse_list

# def print_rmse_score(model):
#     # cv별 프린팅, 평균 저장
#     model_name, score = rmse_cv(model)
#     for i, r in enumerate(score, start=1):
#         print(f'{i} FOLDS: {model_name} RMSLE: {r:.4f}')
#     print(f'\n{model_name} mean RMSLE: {np.mean(score):.4f}')
#     print('='*40)
#     return model_name, np.mean(score)

In [ ]:
# reg = LinearRegression(n_jobs=-1)

In [ ]:
# models = []
# scores = []
# for model in [reg]:
#     model_name, mean_score = print_rmse_score(model)
#     models.append(model_name)
#     scores.append(mean_score)